In [2]:
from analyzer import *

import os
import numpy as np
import librosa
import glob
from sklearn.model_selection import train_test_split
import IPython.display as ipd

In [14]:
import IPython.display as ipd
ipd.Audio(pw2wav(b), rate=16000)

In [11]:
def align_audios_to_spec(wavfile1, wavfile2, canvas_size=2**16):
    iwave, fm = librosa.load(wavfile1, sr=16000)
    lwave, fm = librosa.load(wavfile2, sr=16000)
    iwave_trim, _ = librosa.effects.trim(iwave, top_db=20)
    lwave_trim, _ = librosa.effects.trim(lwave, top_db=20)
    iwave_trim_pad = np.zeros((canvas_size,), dtype=np.float64)
    lwave_trim_pad = np.zeros((canvas_size,), dtype=np.float64)
    if len(iwave_trim) < canvas_size:
        iwave_trim_pad[:len(iwave_trim)] = iwave_trim[:len(iwave_trim)]
    else:
        iwave_trim_pad[:canvas_size] = iwave_trim[:canvas_size]
    if len(lwave_trim) < canvas_size:
        lwave_trim_pad[:len(lwave_trim)] = lwave_trim[:len(lwave_trim)]
    else:
        lwave_trim_pad[:canvas_size] = lwave_trim[:canvas_size]
    iwave_trim_spec = extract(iwave_trim_pad)
    lwave_trim_spec = extract(lwave_trim_pad)
    iwave_pad, lwave_pad = align_specs(iwave_trim_spec, lwave_trim_spec)
    print('wav_signals length', iwave_pad.shape)
    print('label_signals length', lwave_pad.shape)
    return iwave_pad, lwave_pad

In [17]:
np.vstack(t).shape

(1766, 1028)

In [12]:
t = align_audios_to_spec('./data/VCTK-Corpus/wav48/p225/p225_001.wav', './data/VCTK-Corpus/wav48/p226/p226_001.wav')

('wav_signals length', (883, 1028))
('label_signals length', (883, 1028))


In [13]:
ipd.Audio(pw2wav(t[0]), rate=16000)

In [14]:
ipd.Audio(pw2wav(t[1]), rate=16000)

In [41]:
def align_specs(specfile1, specfile2, ndim=1026):
    distance, path = librosa.core.dtw(specfile1[:,:ndim].T, specfile2[:,:ndim].T)
    return specfile1[path[::-1,0],:], specfile2[path[::-1,1],:]

In [42]:
s1, s2 = align_specs(a1, a2)


In [32]:
def save_pair_to(dir_to_pair, map_txt):
    pair_map = []
    with open(map_txt) as fp:
        for line in fp.readlines():
            pair_map.append(line.rstrip().split(','))
    print('{} wavs in the set '.format(len(pair_map)))
    if os.path.exists(dir_to_pair):
        os.system('rm -rf '+dir_to_pair)
    os.system('mkdir -p '+dir_to_pair)
    for wav_pair in pair_map:
        output_filename = os.path.splitext(wav_pair[0])[0].split('/')[-1]+'-'+\
                          os.path.splitext(wav_pair[1])[0].split('/')[-1]
        output_file = os.path.join(dir_to_pair, '{}.bin'.format(output_filename))
        if os.path.exists(output_file):
            print('{} already exists'.format(output_file))
        else:
            print('Process {}'.format(output_file))
            input_spec = np.fromfile(wav_pair[0], np.float32).reshape(-1,1029)
            label_spec = np.fromfile(wav_pair[1], np.float32).reshape(-1,1029)
            features = np.hstack(align_specs(input_spec, label_spec))
            print('features', features.shape)
            with open(output_file, 'wb') as fp:
                fp.write(features.tostring())

In [45]:
def save_pair_to(dir_to_pair, map_txt, final_shape=None):
    pair_map = []
    with open(map_txt) as fp:
        for line in fp.readlines():
            pair_map.append(line.rstrip().split(','))
    print('{} wavs in the set '.format(len(pair_map)))
    if os.path.exists(dir_to_pair):
        os.system('rm -rf '+dir_to_pair)
    os.system('mkdir -p '+dir_to_pair)
    for wav_pair in pair_map:
        output_filename = os.path.splitext(wav_pair[0])[0].split('/')[-1]+'-'+\
                          os.path.splitext(wav_pair[1])[0].split('/')[-1]
        output_file = os.path.join(dir_to_pair, '{}.bin'.format(output_filename))
        if os.path.exists(output_file):
            print('{} already exists'.format(output_file))
        else:
            print('Process {}'.format(output_file))
            input_spec = np.fromfile(wav_pair[0], np.float32).reshape(-1,1029)
            label_spec = np.fromfile(wav_pair[1], np.float32).reshape(-1,1029)
            features = np.hstack(align_specs(input_spec, label_spec))
            if final_shape:
                features_pad = np.zeros(final_shape, np.float32)
                if features.shape[0] < final_shape[0]:
                    features_pad[:features.shape[0],:] = features[:features.shape[0],:]
                else:
                    features_pad[:final_shape[0],:] = input_spec[:final_shape[0],:]
                    label_spec_pad[:final_shape[0],:] = label_spec[:final_shape[0],:]
                features = np.hstack(align_specs(input_spec_pad, label_spec_pad))
            print('features', features.shape)
            with open(output_file, 'wb') as fp:
                fp.write(features.tostring())

In [46]:
save_pair_to('data/matrix_sample/225/pair_pad/train', './data/matrix_sample/225/etc/train_map.txt', (1024,1029*2))

117 wavs in the set 
Process data/matrix_sample/225/pair_pad/train/p226_008-p225_008.bin
('features', (1165, 2058))
Process data/matrix_sample/225/pair_pad/train/p226_001-p225_001.bin
('features', (1072, 2058))
Process data/matrix_sample/225/pair_pad/train/p226_370-p225_061.bin


ValueError: could not broadcast input array from shape (481,1029) into shape (570,1029)

In [34]:
save_pair_to('data/matrix_sample/225/pair/test', './data/matrix_sample/225/etc/test_map.txt')

45 wavs in the set 
Process data/matrix_sample/225/pair/test/p226_019-p225_019.bin
('features', (1389, 2058))
Process data/matrix_sample/225/pair/test/p226_020-p225_020.bin
('features', (688, 2058))
Process data/matrix_sample/225/pair/test/p226_024-p225_024.bin
('features', (1348, 2058))
Process data/matrix_sample/225/pair/test/p226_273-p225_293.bin
('features', (665, 2058))
Process data/matrix_sample/225/pair/test/p226_013-p225_013.bin
('features', (809, 2058))
Process data/matrix_sample/225/pair/test/p226_004-p225_004.bin
('features', (811, 2058))
Process data/matrix_sample/225/pair/test/p226_003-p225_003.bin
('features', (1404, 2058))
Process data/matrix_sample/225/pair/test/p226_292-p225_296.bin
('features', (311, 2058))
Process data/matrix_sample/225/pair/test/p227_228-p225_220.bin
('features', (633, 2058))
Process data/matrix_sample/225/pair/test/p227_317-p225_296.bin
('features', (307, 2058))
Process data/matrix_sample/225/pair/test/p227_400-p225_350.bin
('features', (551, 2058)

In [49]:
glob.glob('./data/matrix_sample/wav48/p*/*wav')

['./data/matrix_sample/wav48/p225/p225_103.wav',
 './data/matrix_sample/wav48/p225/p225_111.wav',
 './data/matrix_sample/wav48/p225/p225_009.wav',
 './data/matrix_sample/wav48/p225/p225_297.wav',
 './data/matrix_sample/wav48/p225/p225_289.wav',
 './data/matrix_sample/wav48/p225/p225_070.wav',
 './data/matrix_sample/wav48/p225/p225_298.wav',
 './data/matrix_sample/wav48/p225/p225_331.wav',
 './data/matrix_sample/wav48/p225/p225_176.wav',
 './data/matrix_sample/wav48/p225/p225_021.wav',
 './data/matrix_sample/wav48/p225/p225_196.wav',
 './data/matrix_sample/wav48/p225/p225_061.wav',
 './data/matrix_sample/wav48/p225/p225_118.wav',
 './data/matrix_sample/wav48/p225/p225_203.wav',
 './data/matrix_sample/wav48/p225/p225_067.wav',
 './data/matrix_sample/wav48/p225/p225_006.wav',
 './data/matrix_sample/wav48/p225/p225_337.wav',
 './data/matrix_sample/wav48/p225/p225_179.wav',
 './data/matrix_sample/wav48/p225/p225_121.wav',
 './data/matrix_sample/wav48/p225/p225_318.wav',
 './data/matrix_samp

In [55]:
def trim_pad_audio(wav_file, sr=16000, top_db=20, duration_in_s=8):
    wave, fm = librosa.load(wav_file, sr=16000)
    wave_trim, _ = librosa.effects.trim(wave, top_db=top_db)
    canvas_size = duration_in_s * sr
    wave_trim_pad = np.zeros((canvas_size,), dtype=np.float64)
    if len(wave_trim) < canvas_size:
        wave_trim_pad[:len(wave_trim)] = wave_trim[:len(wave_trim)]
    else:
        wave_trim_pad[:canvas_size] = wave_trim[:canvas_size]
    return wave_trim_pad

In [90]:
a = np.fromfile("./data/matrix_sample/225//label/p225_017.bin", np.float32).reshape(-1,1029)

In [91]:
import IPython.display as ipd
ipd.Audio(pw2wav(a[:,:-1]), rate=16000)

In [35]:
SP_DIM = 513
FEAT_DIM = SP_DIM + SP_DIM + 1 + 1 + 1

def read_whole_features(file_pattern, num_epochs=1):
    '''
    Return
        `feature`: `dict` whose keys are `sp`, `ap`, `f0`, `en`, `speaker`
    '''
    files = tf.gfile.Glob(file_pattern)
    print('{} files found'.format(len(files)))
    filename_queue = tf.train.string_input_producer(files, num_epochs=num_epochs)
    reader = tf.WholeFileReader()
    key, value = reader.read(filename_queue)
    print("Processing {}".format(key))
    value = tf.decode_raw(value, tf.float32)
    value = tf.reshape(value, [-1, FEAT_DIM])
    return {
        'sp': value[:, :SP_DIM],
        'ap': value[:, SP_DIM : 2*SP_DIM],
        'f0': value[:, SP_DIM * 2],
        'en': value[:, SP_DIM * 2 + 1],
        'speaker': tf.cast(value[:, SP_DIM * 2 + 2], tf.int64),
        'filename': key,
    }

In [39]:
# import tensorflow as tf
# del x
x = read_whole_features('./data/matrix_sample/225/*a*/*bin')

204 files found
Processing Tensor("ReaderReadV2:0", shape=(), dtype=string)


In [40]:
x_all = list()
y_all = list()
f0_all = list()
sv = tf.train.Supervisor()
with sv.managed_session() as sess:
    while True:
        try:
            features = sess.run(x)
            print('Processing {}'.format(features['filename']))
            x_all.append(features['sp'])
            y_all.append(features['speaker'])
            f0_all.append(features['f0'])
        finally:
            pass

x_all = np.concatenate(x_all, axis=0)
y_all = np.concatenate(y_all, axis=0)
f0_all = np.concatenate(f0_all, axis=0)

INFO:tensorflow:Starting standard services.
INFO:tensorflow:Starting queue runners.
Processing ./data/matrix_sample/225/train/p229_022.bin
Processing ./data/matrix_sample/225/label/p225_153.bin
Processing ./data/matrix_sample/225/label/p225_300.bin
Processing ./data/matrix_sample/225/train/p226_023.bin
Processing ./data/matrix_sample/225/label/p225_191.bin
Processing ./data/matrix_sample/225/train/p226_012.bin
Processing ./data/matrix_sample/225/train/p227_267.bin
Processing ./data/matrix_sample/225/train/p228_005.bin
Processing ./data/matrix_sample/225/label/p225_158.bin
Processing ./data/matrix_sample/225/train/p227_012.bin
Processing ./data/matrix_sample/225/train/p228_010.bin
Processing ./data/matrix_sample/225/label/p225_020.bin
Processing ./data/matrix_sample/225/label/p225_005.bin
Processing ./data/matrix_sample/225/train/p228_061.bin
Processing ./data/matrix_sample/225/label/p225_330.bin
Processing ./data/matrix_sample/225/label/p225_314.bin
Processing ./data/matrix_sample/225/

Processing ./data/matrix_sample/225/train/p229_018.bin
Processing ./data/matrix_sample/225/label/p225_349.bin
Processing ./data/matrix_sample/225/train/p227_053.bin
Processing ./data/matrix_sample/225/train/p226_072.bin
Processing ./data/matrix_sample/225/label/p225_109.bin
Processing ./data/matrix_sample/225/label/p225_352.bin
Processing ./data/matrix_sample/225/train/p227_148.bin
Processing ./data/matrix_sample/225/train/p229_009.bin
Processing ./data/matrix_sample/225/train/p228_305.bin
Processing ./data/matrix_sample/225/train/p227_009.bin
Processing ./data/matrix_sample/225/train/p226_151.bin
Processing ./data/matrix_sample/225/label/p225_366.bin
Processing ./data/matrix_sample/225/label/p225_108.bin
Processing ./data/matrix_sample/225/label/p225_021.bin
Processing ./data/matrix_sample/225/train/p227_005.bin
Processing ./data/matrix_sample/225/train/p227_059.bin
Processing ./data/matrix_sample/225/label/p225_010.bin
Processing ./data/matrix_sample/225/train/p229_016.bin
Processing

In [41]:
SPEAKERS = range(225,230)
for s in SPEAKERS:
    print('Speaker {}'.format(s))
    f0 = f0_all[np.int64(s) == y_all]
    print('  len: {}'.format(len(f0)))
    f0 = f0[f0 > 2.]
    f0 = np.log(f0)
    mu, std = f0.mean(), f0.std()
    print mu, std
    # Save as `float32`
    with open('./data/matrix_sample/225/etc/{}.npf'.format(s), 'wb') as fp:
        fp.write(np.asarray([mu, std]).tostring())


# ==== Min/Max value ====
# mu = x_all.mean(0)
# std = x_all.std(0)
q005 = np.percentile(x_all, 0.5, axis=0)
q995 = np.percentile(x_all, 99.5, axis=0)

# Save as `float32`
with open('./data/matrix_sample/225/etc/xmin.npf', 'wb') as fp:
    fp.write(q005.tostring())

with open('./data/matrix_sample/225/etc/xmax.npf', 'wb') as fp:
    fp.write(q995.tostring())


Speaker 225
  len: 44623
5.19598 0.146771
Speaker 226
  len: 22226
4.70987 0.157627
Speaker 227
  len: 23826
4.78033 0.119942
Speaker 228
  len: 19674
5.29525 0.140719
Speaker 229
  len: 16070
5.18284 0.0900351
Speaker 230
  len: 0
nan nan
Speaker 231
  len: 0
nan nan
Speaker 232
  len: 0
nan nan
Speaker 233
  len: 0
nan nan
Speaker 234
  len: 0
nan nan
Speaker 235
  len: 0
nan nan
Speaker 236
  len: 0
nan nan
Speaker 237
  len: 0
nan nan
Speaker 238
  len: 0
nan nan
Speaker 239
  len: 0
nan nan
Speaker 240
  len: 0
nan nan
Speaker 241
  len: 0
nan nan
Speaker 242
  len: 0
nan nan
Speaker 243
  len: 0
nan nan
Speaker 244
  len: 0
nan nan
Speaker 245
  len: 0
nan nan
Speaker 246
  len: 0
nan nan
Speaker 247
  len: 0
nan nan
Speaker 248
  len: 0
nan nan
Speaker 249
  len: 0
nan nan
Speaker 250
  len: 0
nan nan


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice.
  
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
y_all

array([227, 227, 227, ..., 226, 226, 226])